#1. https://python-forum.io/Thread-lambda-or-operator

In [12]:
from operator import methodcaller

In [13]:
def ask_valid1(x, conversion=lambda choice: choice.lower()):
    return conversion(x)

In [14]:
def ask_valid2(x, conversion=methodcaller('lower')):
    return conversion(x)

In [15]:
def ask_valid3(x, conversion=str.lower):
    return conversion(x)

In [16]:
%timeit ask_valid1('Name')

335 ns ± 2.85 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [17]:
%timeit ask_valid2('Name')

353 ns ± 6.49 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [18]:
%timeit ask_valid3('Name')

312 ns ± 18.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


I found some thoughts about this [here](https://treyhunner.com/2018/09/stop-writing-lambda-expressions/).

In my opinion, the third version (`str.lower`) looks more pythonic. 

`methodcaller`
If you look at the operator module sources, you find that `methodcaller` is a light-weight class. In any case, 
using it,  you implicitly include additional code to be executed. This, in turn, can slightly increase the time of execution. 

`str.lower` vs `lambda x: x.lower()`
Using `lambda` keyword is redundant in this case, `str.lower` is shorter. 

Lets look at benchmarks: 


Finally, if you have several methods, 
where the `conversion` argument should have default value (a function), 
I would suggest to define something like `_default_conversion` function, e.g. 

In [19]:
def _default_conversion(x):
    # manage all methods 
    # that include conversion function in one place.
    # Might be also useful when debugging.
    return x.lower()

#OR 

_default_conversion = str.lower
    
class A:

    def ask_valid(self, *args, conversion=_default_conversion):
        pass

    def ask_invalid(self, *args, conversion=_default_conversion):
        pass

#2. https://python-forum.io/Thread-proper-syntax-for-itertuples

You don't need to use loops for this task at all. 

Something like this: 

df.loc[df.loc[:, 'Close'] > df.loc[:, 'prev'], 'trade2'] = '+'
df.loc[df.loc[:, 'Close'] < df.loc[:, 'prev'], 'trade2'] = '-'
df.loc[df.loc[:, 'Close'] == df.loc[:, 'prev'], 'trade2'] = df.loc[((df.loc[:, 'trade2'] =='+')|df.loc[:, 'trade2'] == '-').last_valid_index()]['trade2']

should work. 



#3. https://python-forum.io/Thread-Simple-String-to-Time-within-a-pandas-dataframe

In [1]:
import pandas as pd

In [3]:
df = pd.DataFrame({'atime': ['13-06-2019 10:00', '12-06-2019 09:15'], 'x': [1,2]})

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
atime    2 non-null object
x        2 non-null int64
dtypes: int64(1), object(1)
memory usage: 112.0+ bytes


In [9]:
df.atime = pd.to_datetime(df.atime)

In [10]:
df

,atime,x
0,2019-06-13 10:00:00,1
1,2019-12-06 09:15:00,2


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
atime    2 non-null datetime64[ns]
x        2 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 112.0 bytes


#4. https://python-forum.io/Thread-How-to-vstack-matrixs-in-numpy-with-different-numbers-of-columns

In [22]:
import numpy as np
a = np.array([["A1", "B1", "C1"], ["A1", "B1", "C1"]])
b = np.array([["A2", "B2"], ["A2", "B2"]])
c = np.array([["A3", "B3"], ["A3", "B3"]])
a = pd.DataFrame(a, columns=None)
b = pd.DataFrame(b, columns=None)
pd.concat([a,b]).fillna('').values

array([['A1', 'B1', 'C1'],
       ['A1', 'B1', 'C1'],
       ['A2', 'B2', ''],
       ['A2', 'B2', '']], dtype=object)

In [23]:
import pandas as pd

In [24]:
a = pd.DataFrame(a, columns=None)
b = pd.DataFrame(b, columns=None)

In [25]:
b

,0,1
0,A2,B2
1,A2,B2


In [26]:
pd.concat([a,b]).fillna('').values

array([['A1', 'B1', 'C1'],
       ['A1', 'B1', 'C1'],
       ['A2', 'B2', ''],
       ['A2', 'B2', '']], dtype=object)

#5. https://python-forum.io/Thread-Converting-str-to-binary

In [3]:
mystring = 'My beautiful house'

def word2bin(w):
    return '0b' + ''.join('{0:08b}'.format(ord(x), 'b') for x in w)

bin(sum(map(lambda x: int(x, 2), (word2bin(x) for x in mystring.split()))))

'0b11000100110010101100001011101011101110011011000110111000011011001001010'

#6. https://python-forum.io/Thread-How-to-re-arrange-DataFrame-columns

In [6]:
import pandas as pd

In [7]:
micolumns = pd.MultiIndex.from_tuples([('X', 'foo', '10'), ('X', 'bar', '10'),
                                       ('Y', 'foo', '10'), ('Y', 'bar', '10')],
                                      names=['l0', 'l1', 'l2'])

In [8]:
micolumns

MultiIndex(levels=[['X', 'Y'], ['bar', 'foo'], ['10']],
           labels=[[0, 0, 1, 1], [1, 0, 1, 0], [0, 0, 0, 0]],
           names=['l0', 'l1', 'l2'])

In [30]:
arr = pd.DataFrame(pd.np.arange(12).reshape(3,4), columns=micolumns)

In [16]:
arr

l0   X       Y    
l1 foo bar foo bar
l2  10  10  10  10
0    0   1   2   3
1    4   5   6   7
2    8   9  10  11

In [17]:
arr.reset_index()

l0 index   X       Y    
l1       foo bar foo bar
l2        10  10  10  10
0      0   0   1   2   3
1      1   4   5   6   7
2      2   8   9  10  11

In [29]:
arr.T.reset_index()

,l0,l1,l2,0,1,2
0,X,foo,10,0,4,8
1,X,bar,10,1,5,9
2,Y,foo,10,2,6,10
3,Y,bar,10,3,7,11


#7. https://python-forum.io/Thread-Help-understanding-Bioinformatics-question

You need to divide your problem into a set of small ones. Underlying math isn't quite hard: Jaccard coefficient (similarity) is a fraction of the measure of an intersection of two sets and the measure of a union of them. Jaccard distance seems to be (1 - Jaccard coefficient).

So, you need to implement a function that traverse the specified directory and returns 
data loaded from two files. This function could be implemented as a generator. 
This generator will yield a new pair of data until all possible combinations
be traversed (s(s-1)/2, where s is the number of files in the folder).  


Below is a sketch of the solution; completely not tested but might be helpful...


In [ ]:
def traverse_dir(path='.'):
    # some code goes here, probably you'll need to use os.path.walk
    yield df1, df2, filenames  # df1, df2 assumed to be pandas dataframes; each dataframe has two columns
    
   
    
def get_jaccard(df1, df2, n=3):
    """Return Jaccard distance between two dfs of specified form
    
    Parameters
    ==========
        
        :param df1: Pandas data frame
        :param df2: Pandas data frame
        :param n: an integer, the number of most frequent ... to use
    
    
    Notes
    =====
    
    df1, df2 assumed to have the following form:
    
    df1 = pd.DataFrame({0: ['AACCTTGG', 'CCTTGGA'], 1:[4, 8]})
    df2 = pd.DataFrame({0: ['AACCTTGG', 'CCTTGGA', 'ACC'], 1:[3, 4, 8]})
    
    Pandas assumed to be imported as pd.
    """
    
    d1 = df1.sort_values(by=1, ascending=False)[:n]
    d2 = df1.sort_values(by=1, ascending=False)[:n]
    common = pd.np.intersection1d(d1.iloc[:, 0].values, d2.iloc[:, 0].values)
    a = d1[d1.iloc[:, 0].isin(common) & d2.iloc[:, 0].isin(common)].iloc[:, 1].values
    b = d2[d1.iloc[:, 0].isin(common) & d2.iloc[:, 0].isin(common)].iloc[:, 1].values
    comm_measure = pd.np.vstack([a,b]).min(axis=0).sum()
    all_unique = ... # write something here... 
    
    return  (1 - comm_measure / all_unique)


requested_path = input('Enter path:')
n = input('Enter n:')

# and something like this... 
for a, b, filenames in traverse_dir(requested_path):
    print("Processing files: {}, result={} ".format(filenames, get_jaccard(a, b, n=n)))


In [3]:
import pandas as pd
df1 = pd.DataFrame({0: ['AACCTTGG', 'CCTTGGA'], 1:[4, 8]})
df2 = pd.DataFrame({0: ['AACCTTGG', 'CCTTGGA', 'ACC'], 1:[3, 4, 8]})

,0,1
2,ACC,8
1,CCTTGGA,4
